# From: Transformers and Text-Generation
by Liam Dugan (UPenn). 


Please write your answers and code in the cells with questions below. 

----------

For this homework, we will take ideas from the entire class: language models, text generation, vector-based word representations, syntactic analysis, and neural networks. We'll be using large, pre-trained language models to generate text, and studying how we can fine-tune these large language models to generate text in whatever genre and style we want!

In this assignment you will get:
1. An overview of the "Transformer" architecture is and why it is particularly well suited for Natural Language Processing tasks
2. An introduction to the Generative Pretrained Transformer (GPT) family, which is a set of large-scale language models that can be used to generate text that often sounds like it was written by a human.
3. Experience with using the HuggingFace package to fine-tune these models to generate text that sounds like it comes from a specific source.

# Part 1: What is a Transformer? (Reading)
<figure align="center">
<img src="https://media.giphy.com/media/VeWllmR9zfaco/giphy.gif" />
<figcaption>(It's probably not this guy, right?)</figcaption>
</figure>

### The Transformer

The current state-of-the-art for a variety of natural language processing tasks belongs to the **Transformer** architecture, first published December 6th 2017. 

The Transformer can be thought of as a big feed-forward network with every feed-forward layer containing something called an "attention module". 

>You might be wondering: why are we moving back to feed-forward networks after having so much success with recurrent neural networks and variants like LSTMs? Aren't RNNs naturally poised to handle sequences as their inputs? Well, as it turns out, the sequential nature of RNNs make them really difficult to train in a distributed/parallel fashion. So while RNNs make more sense to use on sequences of inputs, serial networks such as the transformer can be trained much faster, allowing orders of magnitude more training data to be used. 



### Reading \# 1 - What is a Transformer?

In order to get a good grasp on exactly *why* these models are so good it's important to understand what they are and how they work. 

Your first task for this homework is to read the blog post ["The Illustrated Transformer" by Jay Alammar](http://jalammar.github.io/illustrated-transformer/). This blog post explains the transformer architecture (and the all-important "Attention Module") with helpful visualizations and diagrams. 

**You should read this post very closely and understand exactly what the Transformer is and how it works. Once you're finished reading, answer the following questions in 2-3 sentences each.**

1. (2 pts) What is Self-Attention (at a high level)?

   > A model's self-attention mechanism informs the encoder as to the importance of one input sequence components more than another based on learned behavior. Learned behavior is usually associate with producing training data and known labels of correct outcomes.  By seeking to reduce the loss by training the various weight vectors in the model, the model learns behavior.

2. (2 pts) How is Self-Attention computed?

   > The input enbeddings are multiplied by three weight matrixes to produce: vectors, query, key and value vectors. Then a score is created for each word in the input sentence against the word being processed. This is done by producing the dot product of the query vector and the key vector. Divide by 8 (square root of the dimension of the key vector) and pass the data through the softmax function to normalize the score. Scores are all positive and add to one. Then multiply the softmax output by the value vector. Finally, add up all the scores to get the output of the self-attention layer.

3. (2 pts) What do the "Query", "Key", and "Value" vectors encode (at a high level)?

   > Query encodes the representation of the current token in the analysis process. Key encodes a representations of any input word embedding, or the  component from a previous layer, in the input sequence. Word embeddings only occur in the input layer or first encoder. All other encoders, for which the blog post discussed 6 encoders, are stacked and receive input from the layer below. The value vector encodes the contender word representations which we can multiply by a small number (i.e. 0.0001) to wash out less relevant contenders and focus on the better ones.  The names, query, key and value, are better than most at reflecting their function.

4. (2 pts) What is an attention "head" and why should we use multiple heads?

   > It is an independent set of matrixes described in self-attention that allow the system to entertain multiple interpretations and weights for interrelating the input when transforming the input to produce the output. The number of heads determines the multiple interpretations the system can entertain before deciding. I think of attention heads like the quanitity of internal deliberation the system allows.

5. (2 pts) What are positional embeddings?

   > Positional embeddings built in demonstrable distance and order between tokens in the embedding. Many times they are created using the cosine and sine functions because the input can be of any length and these two functions provide a consistent distance output.

6. (2 pts) Why are positional embeddings important?

   > Positional encodings create an embedding for the order of the sequence since to correctly devine the meaning of words order matters, then positional embeddings provide the important function of expressing the order of inputs.


-----------
### Reading \# 2 - Transformer Language Models

On June 11th 2018, OpenAI released a model named **GPT**, standing for *Generative Pre-Trained Transformer*. This model was a Transformer architecture that was modified such that it could be used for Text Generation instead of sequence to sequence modeling. This model was also pre-trained, which means that anyone could download the fully trained model and use it without needing to train the model themselves. 

On February 14th 2019, OpenAI released a blog post detailing a brand new version of GPT that had an insane **1.5 billion parameters**. They named this version **GPT-2**. To train such a large model, OpenAI crawled 40GB worth of text from the web (roughly 20,000,000,000 words). 

GPT-2 is an extremely impressive language model that can generate text that often sounds so plausible that it seems like it might have been written by a human.  Here is an example of what GPT-2 can generate, taken from [OpenAI’s blog post](https://openai.com/blog/better-language-models/):

**Human-Written Prompt:**
> “Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today.”

**Model Continuation:**
> The 19-year-old singer was caught on camera being escorted out of the store by security guards.
> 
> The singer was wearing a black hoodie with the label ‘Blurred Lines’ on the front and ‘Fashion Police’ on the back.
> 
> Scroll down for video
> 
> Shoplifting: Miley Cyrus was caught shoplifting from Abercrombie and Fitch on Hollywood Boulevard today (pictured)
> 
> The singer was also wearing a pair of black-rimmed glasses, a black jacket, black jeans and black sandals.
> 
> She was carrying a pair of black and white striped gloves and a small black bag.

*(To be clear: Miley Cyrus was never actually arrested for shoplifting)*.  

The quality of the model's output was so good, that the researchers at OpenAI were worried that, in addition to positive applications, it might be used for malicious purposes like generating fake news, impersonating people online, creating abusive or fake content to post on social media, or automating spam and phishing scams. In a [demo to Wired Magazine](https://www.wired.com/story/ai-text-generator-too-dangerous-to-make-public/), the reporter typed in “Hillary Clinton and George Soros” and GPT-2 generated a crazy rant with lots of conspiracy theories: 
> Hillary Clinton and George Soros are a perfect match, and their agenda appears to be to create a political movement where Soros and his political machine and Clinton are two of the only major players. This is the first time Soros and Clinton have been caught on tape directly colluding in promoting the same false narrative. One of the key revelations in the leaked audio was Clinton's admission to a Russian banker that she knew about the Uranium One deal before it was approved by Congress. Clinton was shown sharing the same talking points that were originally drafted by a Fusion GPS contractor hired by an anti-Trump Republican donor. The leaked audio is the clearest evidence yet that the Clinton campaign and the Hillary Foundation colluded with Fusion GPS to manufacture propaganda against President Trump.


They were concerned enough that they labeled GPT-2 "too dangerous to release", and OpenAI initially refused to release their dataset, training code, or GPT-2 model weights.  OpenAI decided to release in a delayed, phased fashion so that researchers could spend time working on automatic detection of generated text.

In this homework, you'll get to be the judge of how good GPT-2 is, as you'll be using it yourself to generate text!

**To start your journey into the world of Text Generation, you should read Part 1 of the blog post ["The Illustrated GPT-2" by Jay Alammar](http://jalammar.github.io/illustrated-gpt2/) and answer the following questions in 2-3 sentences each**

7. (4 pts) How does the architecture of GPT-2 differ from the standard Encoder-Decoder Transformer model?
   > GPT-2 focuses is a Transformer-Decoder model which means that where a standard architecture has encoders and decoders, GPT-2 has only decoders. The decoders do away with the second layer of self-attention used in standard decoder that are part of encoder/decoder transformers. The system goes back to the only language model behavior of only generating one next token at a time. Further, GPT-2 uses auto-regression which feeds the next token generated back to the model as input for deciding the next token after that. 

8. (4 pts) What is the difference between "Masked Self-Attention" and "Self-Attention"
   > GPT-2 uses masked self-attention which means that it only looks at the tokens before the next token being generated rather than self-attention. Models like Bert use self-attention which allows the system to look at tokens before and after the next token being generated.  The information in masked self-attention models is not blanked out by a [masked] token; instead, it prevents itself from looking to the right of token being generated next. Imagine a translation task where a whole sentence is being translated. In this case, the data is available to the right of the token being generated and GPT-2 simple does not use it.
   
9. (4 pts) What are logits? How are they computed? and How does GPT-2 use them to decide which word to predict next?
   > Logits are the output token probabilities. Computed by, the output vector of the last decoder is multiplied by the token embeddings for the vocabulary. These scores come out of the self-attention layer's softmax function and turn into the proability of a given word in the vocabulary being the next best word to output. The word with the highest probability could always be chosen if using a greedy methodology. Many models samples the top few next word probabilities and randomely selects one to avoid cyclical input-output loops. 

### Aside: GPT-3 

On June 11th 2020, OpenAI released GPT-3 [(paper)](https://arxiv.org/pdf/2005.14165.pdf) [(wikipedia)](https://en.wikipedia.org/wiki/GPT-3). This model has an unfathomable **175 billion parameters** (100x larger than GPT-2!) and was trained on 570GB of text! This model is virtually indistinguishable from human output and can generate text about any topic and in any style with only a few words of priming text. It can do some very terrifying things.

GPT-3 Can:
- Generate JSX code off natural language descriptions
- Generate Emojis based off of descriptions of the feeling
- Generate regular expressions off natural language descriptions
- Generate website mockups off natural language descriptions
- Generate charts with titles, labels and legends from natural language descriptions
- Explain python code in plain english
- Automatically generate quiz questions (and grade them)
- Generate Latex from natural language descriptions
- Generate Linux commands from natural language descriptions
- Generate a Machine Learning model from natural language descriptions

[Here's a collection of 21 things GPT-3 can do (with examples)](https://machinelearningknowledge.ai/openai-gpt-3-demos-to-convince-you-that-ai-threat-is-real-or-is-it/#OpenAI_GPT-3_Demos)

[Here's a NYT article about how GPT-3 can write code, poetry, and argue](https://www.nytimes.com/2020/11/24/science/artificial-intelligence-ai-gpt3.html)

[Here's an article GPT-3 wrote for The Guardian about how it loves humans and would never subjugate humanity](https://www.theguardian.com/commentisfree/2020/sep/08/robot-wrote-this-article-gpt-3)

**You may optionally choose to read Jay Alammar's most recent blog post ["How GPT3 Works - Visualizations and Animations"](http://jalammar.github.io/how-gpt3-works-visualizations-animations/) from July 2020 if you're curious as to how GPT-3 differs from GPT-2**

Similarly to GPT-2, OpenAI has decided not to release GPT-3, this time opting to put GPT-3 behind an API which you need to request permission to use. This allows them to control exactly who can generate text and what type of text is generated. While this is a good solution in the short term, the long term implications of GPT-3 are still unclear.

If you are interested in trying out GPT-3 yourself, feel free to [Join the OpenAI API Waitlist](https://share.hsforms.com/1Lfc7WtPLRk2ppXhPjcYY-A4sk30)

-------------------------------

# Part 2: GPT-2 Text Generation with HuggingFace

Phew, that was a lot of reading. Now lets get to the fun part! Let's use the transformer to generate some text!!

We will use the [Transformers library from HuggingFace](https://transformer.huggingface.co), which provides support for many Transformer-based language models like GPT-2. 

**IMPORTANT: Make sure that you have GPU set as your Hardware Accelerator in `Runtime > Change runtime type` before running this Colab.**

In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## 2.1 The 'Pipeline' Interface

The simplest way to use the HuggingFace library is to use their [Pipeline interface](https://huggingface.co/transformers/main_classes/pipelines.html)

There are many different types of Pipelines available but in this section we'll use the TextGenerationPipeline to get up and running with pretrained gpt2 as fast as possible

In [2]:
from transformers import pipeline

In [3]:
# Note: device=0 means to use GPU, device=-1 is to use CPU
generator = pipeline('text-generation', model='gpt2', device=0) 

In [5]:
outputs = generator('I wonder what I will generate?')
print(outputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': "I wonder what I will generate? It's not like every game is a challenge and as a result you may have to work hard to try and make something fun. However, I wouldn't be able to guarantee that it will be something you try out"}]


Note that the 'text-generation' pipeline will work with any **auto-regressive** language model (a.k.a 'causal-lm' models according to the HuggingFace lingo). You can find a list of all such models here https://huggingface.co/models?filter=causal-lm. 

10. (6 pts) **Your first task is to use the Pipeline interface to get generation output below for at least two different 'causal-lm' models (One of these two can be a different version of GPT2, but make sure at least one is a non-gpt family language model)**

In [ ]:
## YOUR CODE HERE FOR MODEL 1

In [14]:
# Note: device=0 means to use GPU, device=-1 is to use CPU
generator = pipeline('text-generation', model='Moxis/Harry_Potter_text_generation', device=0) 
outputs = generator('I wonder what I will generate?')
print(outputs)

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.44G [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/290 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/51.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/127 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I wonder what I will generate? ”'}]


In [23]:
outputs = generator('I wonder what I will generate?')
print(outputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I wonder what I will generate? No, don’t worry.'}]


In [12]:
## YOUR CODE HERE FOR MODEL 2
generator = pipeline('text-generation', model='achrekarom/text_generation', device=0)


Downloading:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/551M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/767 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [13]:
outputs = generator('I wonder what I will generate?')
print(outputs)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'I wonder what I will generate?\n\nI have a lot of questions about the future of the'}]


## 2.2 Dissecting the Pipeline
Now that was easy!

As beautiful and easy as the Pipeline interface is, we want to know what's going on under the hood!

There are four main steps to a text generation pipeline:
1. (Tokenize) Turn the raw input text into a vector of integer token IDs using a tokenizer

2. (Encode) Feed those token IDs into the language model by querying for each token's embedding in the model's embedding matrix (the "encoder") and then feed the "encoded" sequence into the decoder module

3. (Decode) The decoder will output logits (a probability distribution over all possible integer token IDs) and we sample from those logits to get our next token -- repeat until EOS token is generated or we hit max_length

4. (Detokenize) Take the output sequence of token IDs and turn them from integer token IDs back to tokens with the tokenizer

Below you'll see how HuggingFace does this:

First we have to initialize both the tokenizer and the model from their pre-trained checkpoints. Note that the tokenizer has to match the model.

In [24]:
from transformers import GPT2Tokenizer, GPT2LMHeadModel# AutoTokenizer, AutoModelForCausalLM

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()

In [25]:
#### Step 1: Tokenize the input into integer token IDs
inputs = tokenizer.encode("Hello, how are you?", return_tensors='pt').to(model.device)
print("Input Token IDs: " + str(inputs))

# Input Token IDs: tensor([[15496,    11,   703,   389,   345,    30]], device='cuda:0')

Input Token IDs: tensor([[15496,    11,   703,   389,   345,    30]], device='cuda:0')


In [ ]:
#### Step 2 and 3: Feed in the integer token IDs and get out a sequence of token IDs as output
outputs = model.generate(inputs)
print("Output Token IDs: " + str(outputs))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output Token IDs: tensor([[15496,    11,   703,   389,   345,    30,   198,   198,    40,  1101,
           257,  1310,  1643,   286,   257, 34712,    13,   314,  1101,   257]],
       device='cuda:0')


/usr/local/lib/python3.7/dist-packages/transformers/generation_utils.py:1364: UserWarning: Neither `max_length` nor `max_new_tokens` has been set, `max_length` will default to 20 (`self.config.max_length`). Controlling `max_length` via the config is deprecated and `max_length` will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  UserWarning,


In [26]:
#### Step 2 and 3: Feed in the integer token IDs and get out a sequence of token IDs as output
outputs = model.generate(inputs)
print("Output Token IDs: " + str(outputs))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Output Token IDs: tensor([[15496,    11,   703,   389,   345,    30,   198,   198,    40,  1101,
           257,  1310,  1643,   286,   257, 34712,    13,   314,  1101,   257]],
       device='cuda:0')


In [27]:
#### Step 4: Feed in the integer token IDs and get out a sequence of token IDs as output
output_text = [tokenizer.decode(x) for x in outputs]
print("Output Text: " + str(output_text))

Output Text: ["Hello, how are you?\n\nI'm a little bit of a nerd. I'm a"]


Now that you have dissected the pipeline, it's time to play with some common parameters!

[Check out this demo notebook from HuggingFace](https://github.com/huggingface/blog/blob/master/notebooks/02_how_to_generate.ipynb) for a good overview of the different generation parameters and what they do (with example code!).

The full documentation on all of the parameters you can use in the generate function can be found [here](https://huggingface.co/transformers/main_classes/model.html#transformers.generation_utils.GenerationMixin.generate)

As an example, below we have a call to generate that:
- randomly samples from the top 50 words in the output distribution (rather than just greedily picking the best one every time)
- downweights the probability of all previously generated tokens by a factor of 1.2 (to prevent repetition)
- goes on for 512 tokens, because its more interesting

In [28]:
inputs = tokenizer.encode("Hello, how are you?", return_tensors='pt').to(model.device)
outputs = model.generate(
      inputs,
      do_sample=True,          # Randomly sample from the logits instead of greedily picking next word with highest probability
      top_k=50,                 # Only sample from the top 50 most likely words
      repetition_penalty=1.2,    # Downweights the probability of all previously generated tokens by a factor of 1.2
      max_length=512          # Generate for a maximum of 512 tokens
  )
print([tokenizer.decode(x) for x in outputs][0])


The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


Hello, how are you?
This man went to bed two hours ago. I had no idea where he left it all off but we've been living on the floor next door for 12 months without even opening my mouth yet - this guy lives in North Carolina with his wife and four kids (she's an awesome horticulturalist), so any thought that will come up is of him or not really good luck! Can your husband get away here knowing what happened when they did these things? It would be hard because one can't just say "we found life like before" if after some time someone doesn´t know who/what actually got caught trying to do anything wrong; there have already fallen prey as such throughout our land! You cannot ignore why people don�re working overtime looking down their noses into a water hole while nobody ever tries something else. We hope others see through those sadistic eyes though.. This may very well end badly due neither party cares about anybody... They could use them too lol<|endoftext|>


**11. Your job is to provide two different examples of generation output from GPT-2 with different choices of generation parameters. You must also provide a 1-2 sentence explanation of what these parameters do and how they affect your output**

Feel free to get creative with this! Really poke around and try to find the combination of settings that gives you the best sounding text! The ways in which these parameters affect how 'human-like' a section of generated text sounds is an area of active research. :)

In [ ]:
## YOUR CODE HERE FOR HYPERPARAMETER VARIATION 1

(4 pts) YOUR ANSWER HERE - EXPLANATION FOR HPARAM VARIATION 1

In [ ]:
## YOUR CODE HERE FOR HYPERPARAMETER VARIATION 2

(4 pts) YOUR ANSWER HERE -- EXPLANATION FOR HPARAM VARIATION 2

## 2.3 Fine-Tuning GPT-2
Okay now time for the best part!

Generating general-purpose text from pre-trained models is great, but what if we want our text to be in a specific genre or style? Luckily for us, the GPT family of models use the idea of "Transfer learning" -- using knowledge gained from one problem (or training setting), and applying it to another area or domain. The idea of transfer learning for NLP, is that we can train a language model on general texts, and then adapt it to use it for a specific task or domain that we're interested in. This process is also called **fine-tuning**.

In this section we'll walk you through an example of using HuggingFace to fine-tune GPT-2 and then you'll be asked to fine-tune GPT-2 on two datasets of your own choosing!

### Fine-Tuning Example using HuggingFace Datasets library: Crime and Punishment

For our fine-tuning example we're going to train GPT-2 to mimic the style of Fyodor Dostoevsky's novel "Crime and Punishment"

We will be downloading our data using the HuggingFace [Datasets](https://huggingface.co/docs/datasets/) library.

In [ ]:
!pip install datasets

In [ ]:
from transformers import Trainer, TrainingArguments, DataCollatorForLanguageModeling
import datasets
from datasets import load_dataset, list_datasets

### Step 1: Initialize a Brand New GPT-2 Model and Tokenizer

In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('gpt2')
tokenizer.pad_token = tokenizer.eos_token
model = GPT2LMHeadModel.from_pretrained('gpt2').cuda()
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

###Step 2: Load the text of "Crime and Punishment" and tokenize it

The 'load_dataset' function queries for a dataset with a certain tag and downloads the corresponding data from HuggingFace's hosting site. This allows us to download all sorts of datasets through the same interface!

The documentation for load_dataset can be found [here](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset)

Here we take our tokenizer and run it on the entirety of Crime and Punishment in a single batch by using map on our custom encode function.

In [ ]:
def encode(batch): return tokenizer([x.strip('\n\r') for x in batch['line']], truncation=True, padding=True)

crime_and_punishment = load_dataset('crime_and_punish', split='train')
processed = crime_and_punishment.map(encode, batched=True, batch_size=len(crime_and_punishment))
processed.set_format('torch', columns=['input_ids', 'attention_mask'])

### Step 3: Initialize the Trainer

The 'Trainer' module is the main way we perform fine-tuning. In order to initialize a Trainer, you need a model, tokenizer, TrainingArguments, your training data (in a Dataset object) and something called a data_collator (which tells the Trainer not to look for a vector of labels). 

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/',
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    logging_steps=100,
    weight_decay=0.01,
    logging_dir='./logs',
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=training_args,
    data_collator=data_collator,
    train_dataset=processed,
)

### Step 4: Fine-Tune the Model!

Now we're done! All we have to do is hit run and sit back!

In [ ]:
trainer.train()

### Step 5: Save the Model and use it to Generate!

Save your fine-tuned model and compare its output with regular GPT-2's output to see the difference for yourself!

In [ ]:
trainer.save_model('./dostoevskypt2')

In [ ]:
dostoevskypt2 = pipeline('text-generation', model='./dostoevskypt2', device=0)
gpt2 = pipeline('text-generation', model='gpt2', device=0)

In [ ]:
print(dostoevskypt2('Saint Petersburg is'))
print(gpt2('Saint Petersburg is'))

## PERPLEXITY

12. (2 pts) Using the pointer [here](https://huggingface.co/transformers/perplexity.html), compute the perplexity of the GPT2 pre-trained model on the Wikipedia test set (you can keep the same hyperparameters as in the link) 

In [ ]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF GPT2 ON WIKIPEDIA TEST SET 

# ANSWERS BELOW:
# Load wiki test set
from datasets import load_dataset
import torch
from tqdm import tqdm

test = load_dataset("wikitext", "wikitext-2-raw-v1", split="test")
encodings = tokenizer("\n\n".join(test["text"]), return_tensors="pt")
max_length = model.config.n_positions
stride = 512

# Define a function for ppl
def ppl(model, input_ids_all, stride):
  nlls = []
  for i in tqdm(range(0, input_ids_all.size(1), stride)):
      begin_loc = max(i + stride - max_length, 0)
      end_loc = min(i + stride, input_ids_all.size(1))
      trg_len = end_loc - i  # may be different from stride on last loop
      input_ids = input_ids_all[:, begin_loc:end_loc].to("cuda:0")
      target_ids = input_ids.clone()
      target_ids[:, :-trg_len] = -100

      with torch.no_grad():
          outputs = model(input_ids, labels=target_ids)
          neg_log_likelihood = outputs[0] * trg_len

      nlls.append(neg_log_likelihood)

  ppl = torch.exp(torch.stack(nlls).sum() / end_loc)
  return ppl
ppl(model, encodings.input_ids, stride)

> YOUR PERPLEXITY ANSWER HERE: 87.48 (-20, +10 are fine, beyond that give partial credit, deducting 0.5 as it gets worse)

13. (2 pts) Compute the  perplexity of the dostoevskypt2 model on Wikipedia test set




In [ ]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF DOSTOEVSKYPT2 ON WIKIPEDIA TEST SET

> YOUR PERPLEXITY ANSWER HERE

14. (2 pts) Compute the perplexity of the GPT2 pre-trained model on the Crime and Punishment train dataset

In [ ]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF GPT2 ON CRIME AND PUNISHMENT TRAIN DATASET 

> YOUR PERPLEXITY ANSWER HERE

15. (2 pts) Compute the **train** perplexity of the **dostoevskypt2** model 




In [ ]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF DOSTOEVSKYPT2 ON CRIME AND PUNISHMENT TRAIN DATASET 

> YOUR PERPLEXITY ANSWER HERE



> (1 pt) Which model performs better on Crime and Punishment train set, vanilla GPT-2 or your dostoevskypt2 checkpoint?

> YOUR ANSWER HERE

16. (2 pts) Compute perplexity of the GPT2 model on your raw pride and prejudice text.

In [ ]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF GPT2 ON PRIDE AND PREJUDICE TEXT 

> YOUR PERPLEXITY ANSWER HERE

17. (2 pts) Compute perplexity of the **dostoevskypt2** model on your raw pride and prejudice text.

In [ ]:
## YOUR CODE HERE - FOR COMPUTING PERPLEXITY OF dostoevskipt2 ON PRIDE AND PREJUDICE TEXT 

> YOUR PERPLEXITY ANSWER HERE

### Now's Your Turn!

**Your job is to fine-tune GPT2 one more time with your choice of fine-tuning dataset.**

*****For the fine-tuned model you create, you should clearly demonstrate (through visible generation outputs and analysis) that your fine-tuned model follows the desired style better than vanilla GPT2** ***

Please make sure to give a brief description 

In order to see which datasets are available for download, run the cell below. Pick one that you think would be interesting!

In [ ]:
datasets_list = list_datasets()
print(', '.join(dataset for dataset in datasets_list))

### Tips
- Most of the datasets hosted by HuggingFace are not meant for Causal LM fine-tuning. Make sure you preprocess them accordingly if you want to use them.
- In order to check out information about a dataset hosted by huggingface you can use [this web viewer](https://huggingface.co/datasets/viewer/?dataset=crime_and_punish). Try to avoid downloading a dataset that's too big!
- You will likely have to change the custom 'encode' function for each new dataset you want to fine-tune on. You need to change batch['line'] to instead index with the correct column label for your specific dataset (it probably wont be called 'line').

### Useful Links
[load_datasets Documentation](https://huggingface.co/docs/datasets/package_reference/loading_methods.html#datasets.load_dataset)

[Trainer Documentation](https://huggingface.co/transformers/main_classes/trainer.html#id1)

[Example: Fine-Tuning BERT for Esperanto](https://colab.research.google.com/github/huggingface/blog/blob/master/notebooks/01_how_to_train.ipynb#scrollTo=zTgWPa9Dipk2)

[Example: Fine-Tuning for IMDb Classification](https://colab.research.google.com/drive/1-JIJlao4dI-Ilww_NnTc0rxtp-ymgDgM?usp=sharing#scrollTo=5DEWNilys9Ty)


#### 18. Dataset \#1

In [ ]:
## YOUR CODE HERE - FOR FINE-TUNING GPT2 ON DATASET 

(4 pts) YOUR ANSWER HERE - BRIEF DESCRIPTION OF THE DATASET YOU CHOSE

In [ ]:
## YOUR CODE HERE - FOR GENERATION WITH YOUR FINE-TUNED MODEL AND COMPARISON WITH REGULAR GPT2

(5 pts) YOUR ANSWER HERE - COMPARISON OF YOUR DATASET'S FINE-TUNED OUTPUT VS NON-FINE-TUNED OUTPUT 
